<h2 style="text-align:center;font-size:200%;">
    <b>Healthy Recipe Recommendation</b>
</h2>
<h3  style="text-align:center;">Keywords :
    <span style="border-radius:7px;background-color:limegreen;color:white;padding:7px;">Health</span>
    <span style="border-radius:7px;background-color:limegreen;color:white;padding:7px;">Food</span>
    <span style="border-radius:7px;background-color:limegreen;color:white;padding:7px;">Recommendation</span>
    <span style="border-radius:7px;background-color:limegreen;color:white;padding:7px;">Optimization</span>
    <span style="border-radius:7px;background-color:limegreen;color:white;padding:7px;">PuLP</span>
</h3>

<hr>

<h2 style="font-size:150%;"><span id='top'>Table of Contents</span></h2>
<blockquote>
    <ol>
        <li><a href="#Overview">Overview</a></li>
        <ul>
            <li><a href="#About-the-Dataset">About the Dataset</a></li>
            <li><a href="#What-You-Get-from-this-Notebook">What You Get from this Notebook</a></li>
        </ul>
        <li><a href="#Preparation">Preparation</a></li>
        <li><a href="#Data-Load">Data Load</a></li>
        <li><a href="#Pre-Processing">Pre-Processing</a></li>
        <ul>
            <li><a href="#Delete-Null-Rows-in-Certain-Columns">Delete Null Rows in Certain Columns</a></li>
            <li><a href="#Extract-Certain-Columns">Extract Certain Columns</a></li>
            <li><a href="#Add-Recipe-Information">Add Recipe Information</a></li>
        </ul>
        <li><a href="#EDA">EDA</a></li>
        <ul>
            <li><a href="#Basic-Information">Basic Information</a></li>
            <li><a href="#Rating">Rating</a></li>
            <li><a href="#Nutrients">Nutrients</a></li>
            <li><a href="#Flags">Flags</a></li>
            <li><a href="#Other-Information">Other Information</a></li>
        </ul>
        <li><a href="#Recommendation">Recommendation</a></li>
        <ul>
            <li><a href="#Recommendation-Function">Recommendation Function</a></li>
            <li><a href="#Nutrients-Condition">Nutrients Condition</a></li>
            <ul>
                <li><a href="#Male">Male</a></li>
                <li><a href="#Female">Female</a></li>
            </ul>
            <li><a href="#Recipes-for-YOU">Recipes for YOU</a></li>
            <ul>
                <li><a href="#Adult-Male">Adult Male</a></li>
                <li><a href="#Adult-Female">Adult Female</a></li>
                <li><a href="#Man-in-Summer">Man in Summer</a></li>
                <li><a href="#Man-in-Winter">Man in Winter</a></li>
                <li><a href="#Man-at-Breakfast">Man at Breakfast</a></li>
                <li><a href="#Man-with-Low-Calory">Man with Low Calory</a></li>
                <li><a href="#Man-in-Busy-Day">Man in Busy Day</a></li>
            </ul>
        </ul>
        <li><a href="#References">References</a></li>
    </ol>
</blockquote>

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>

# Overview
## About the Dataset
- 20k recipes listed by recipe **rating**, nutritional information and assigned categories
- Recipes extracted from [Epicurious ](http://www.epicurious.com/recipes-menus)
- Kaggle Dataset Page : [Epicurious - Recipes with Rating and Nutrition](https://www.kaggle.com/datasets/hugodarwood/epirecipes)

## What You Get from this Notebook
- EDA & Visualization
- Mathematical Optimization
    - Python Optimization Library : [PuLP](https://coin-or.github.io/pulp/)
    - In this notebook, I applied **mathematical optimazation** to **food recommendation**
- Food Recommendation
    - [Recommendation of healthy food](#Recommendation-Function) tailored to the each person's health status
    - Foods were recommended based on [official standard nutritions value](#Nutrients-Condition) (Japan)
    - Optimal foods for some situations
        - Seasons ([Summer](#Man-in-Summer), [Winter](Man-in-Winter))
        - Mealtime ([Breakfast](#Man-at-Breakfast), Dinner)

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>

# Preparation

In [ ]:
import numpy as np
import pandas as pd
import os
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
import json
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
import pulp as pl
import warnings
warnings.simplefilter(action='ignore')

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>

# Data Load

In [ ]:
df = pd.read_csv("/kaggle/input/epirecipes/epi_r.csv")
df.head(3)

In [ ]:
df.shape

In [ ]:
for i,j in enumerate(df.columns):
    print(f"{i} : {j}")

In [ ]:
with open("/kaggle/input/epirecipes/full_format_recipes.json","r") as f:
    full_recipe = json.load(f)

In [ ]:
len(full_recipe)

In [ ]:
print(json.dumps(full_recipe[0:2], indent=2))

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>

# Pre-Processing

## Delete Null Rows in Certain Columns

In [ ]:
df[["title","rating","calories","protein","fat","sodium"]].head(3)

In [ ]:
df_p1 = df[df[["title","rating","calories","protein","fat","sodium"]].isnull().any(axis=1)==False].reset_index(drop=True)
df_p1.head()

In [ ]:
df_p1.shape

## Extract Certain Columns

In [ ]:
_certain_cols = ["title","rating","calories","protein","fat","sodium","22-minute meals","advance prep required","alcoholic","bread","breakfast","dinner","drink","drinks",\
                 "egg","fall","fat free","healthy","high fiber","low cal","low carb","low cholesterol","low fat","low sodium","low sugar","low/no sugar","lunch",\
                "meat","quick & easy","quick and healthy","rice","spring","summer","vegetable","winter"]

In [ ]:
df_p2 = df_p1[_certain_cols]

## Add Recipe Information

In [ ]:
full_recipe[0]

In [ ]:
def get_recipe_info(recipe_title):
    recipe_dic = [i for i in full_recipe if "title" in i.keys() and i["title"]==recipe_title]
    if len(recipe_dic) == 0:
        return pd.Series({"len_directions":None, "len_ingredients":None})
    else:
        recipe_dic = recipe_dic[0]
    len_directions = len(recipe_dic["directions"])
    len_ingredients = len(recipe_dic["ingredients"])
    return pd.Series({"len_directions":len_directions, "len_ingredients":len_ingredients})

In [ ]:
df_p2[["len_directions","len_ingredients"]] = df_p2.title.parallel_apply(lambda x : get_recipe_info(x))

In [ ]:
df_p2.head(3)

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>

# EDA

## Basic Information

Number of Recipes

In [ ]:
len(df_p2.title.unique())

Columns

In [ ]:
list(df_p2.columns)

## Rating

average rating

In [ ]:
df_p2.rating.mean()

In [ ]:
hv.Distribution(df_p2.rating).opts(opts.Distribution(title="Distribution of Rating",tools=['hover'],show_grid=True)) + \
hv.BoxWhisker(df_p2.rating).opts(opts.BoxWhisker(title="Rating BoxPlot",tools=['hover'],show_grid=True))

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>

## Nutrients

average nutrients

In [ ]:
df_p2[["calories","protein","fat","sodium"]].mean()

In [ ]:
g1 = hv.BoxWhisker(df_p2.calories).opts(opts.BoxWhisker(title="calories"))
g2 = hv.BoxWhisker(df_p2.protein).opts(opts.BoxWhisker(title="protein"))
g3 = hv.BoxWhisker(df_p2.fat).opts(opts.BoxWhisker(title="fat"))
g4 = hv.BoxWhisker(df_p2.sodium).opts(opts.BoxWhisker(title="sodium"))

(g1+g2+g3+g4).opts(opts.BoxWhisker(width=300, ylabel="value",tools=['hover'],show_grid=True)).opts(shared_axes=False).cols(2)

## Flags

In [ ]:
tmp = df_p2.drop(["title","rating","calories","protein","fat","sodium","len_directions","len_ingredients"], axis=1)
p = pd.DataFrame()
for c in tmp.columns:
    _tmp = tmp[c].value_counts(normalize=True) * 100
    p = pd.concat([p,_tmp], axis=1)
t = pd.melt(p.reset_index(), ['index'])

g1 = hv.Bars(t.query("variable in ['bread','rice','meat','egg','drink','drinks','alcoholic','vegetable']"), ['variable','index'], 'value').opts(title='Ingredients')
g2 = hv.Bars(t.query("variable in ['spring','summer','fall','winter']"), ['variable','index'], 'value').opts(title='Season')
g3 = hv.Bars(t.query("variable in ['breakfast','lunch','dinner']"), ['variable','index'], 'value').opts(title='Eat Timing')
g4 = hv.Bars(t.query("variable in ['22-minute meals','advance prep required','quick & easy','quick and healthy']"), ['variable','index'], 'value')\
    .opts(title='Cooking Categories')
g5 = hv.Bars(t.query("variable in ['healthy','fat free','high fiber','low cal','low carb','low cholesterol','low fat','low sodium','low sugar','low/no sugar']"), \
             ['variable','index'], 'value').opts(title='Recipe Categories')

(g1+g2+g3+g4+g5)\
    .opts(opts.Bars(width=250, height=400,invert_axes=True,stacked=True,legend_position='bottom',tools=['hover'],show_grid=True,xlabel="",ylabel="",fontsize={'title':10}))\
    .opts(shared_axes=False).cols(3)

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>

## Other Information

In [ ]:
df_p2[["len_directions","len_ingredients"]].mean()

In [ ]:
g1 = hv.BoxWhisker(df_p2.len_directions).opts(opts.BoxWhisker(title="directions"))
g2 = hv.BoxWhisker(df_p2.len_ingredients).opts(opts.BoxWhisker(title="ingredients"))

(g1+g2).opts(opts.BoxWhisker(width=300, ylabel="value",tools=['hover'],show_grid=True)).opts(shared_axes=False).cols(2)

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>

# Recommendation

## Recommendation Function

In [ ]:
def recipe_recommend(df, number_of_dishes, number_of_candidates, nut_conf):
    tmp = df.copy()
    candidates_list = []

    # create recommend-list {number_of_candidates} times
    for i in range(0,number_of_candidates):
        m = pl.LpProblem(sense=pl.LpMaximize)
        tmp.loc[:,'v'] = [pl.LpVariable('x%d'%i, cat=pl.LpBinary) for i in range(len(tmp))]

        # maximize rating
        m += pl.lpDot(tmp["rating"], tmp["v"])

        # limit the number of recipes
        m += pl.lpSum(tmp["v"]) <= number_of_dishes

        # limit the calories
        m += pl.lpDot(tmp["calories"], tmp["v"]) >= nut_conf["cal_lo"]
        m += pl.lpDot(tmp["calories"], tmp["v"]) <= nut_conf["cal_up"]

        # limit the protein
        m += pl.lpDot(tmp["protein"], tmp["v"]) >= nut_conf["pro_lo"]
        m += pl.lpDot(tmp["protein"], tmp["v"]) <= nut_conf["pro_up"]

        # limit the fat (sum of fat / calories)
        m += pl.lpDot(tmp["fat"], tmp["v"]) >= nut_conf["fat_lo"]
        m += pl.lpDot(tmp["fat"], tmp["v"]) <= nut_conf["fat_up"]

        # limit the sodium
        m += pl.lpDot(tmp["sodium"], tmp["v"]) >= nut_conf["sod_lo"]
        m += pl.lpDot(tmp["sodium"], tmp["v"]) <= nut_conf["sod_up"]

        m.solve(pl.PULP_CBC_CMD(msg=0, options=['maxsol 1']))

        if m.status == 1:
            tmp.loc[:,'val'] = tmp["v"].apply(lambda x: pl.value(x))
            ret = tmp.query('val==1')["title"].values
            candidates_list.append(ret)
            # update dataframe (remove recommended title )
            tmp = tmp.query('val==0')

    return candidates_list

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>

## Nutrients Condition
consider upper/lower nutrients for male / female  
https://www.otsuka.co.jp/cmt/nutrition/1day/

### Male

In [ ]:
# Daily
male_japanese_day = {
    "cal_up" : 2700,
    "cal_lo" : 2600,
    "pro_up" : 65,
    "pro_lo" : 60,
    "fat_up" : 30,
    "fat_lo" : 20,
    "sod_up" : 7500,
    "sod_lo" : 7000
}
# Daily => 3 x meals
## 1 meal => Daily / 3
male_japanese_each = { k:v/3 for k, v in male_japanese_day.items() }

print("### Nutrients Conditions ###")
print(f"{np.round(male_japanese_each['cal_lo'],1)} =< calory <= {np.round(male_japanese_each['cal_up'],1)}")
print(f"{np.round(male_japanese_each['pro_lo'],1)} =< protain <= {np.round(male_japanese_each['pro_up'],1)}")
print(f"{np.round(male_japanese_each['fat_lo'],1)} =< fat <= {np.round(male_japanese_each['fat_up'],1)}")
print(f"{np.round(male_japanese_each['sod_lo'],1)} =< sodium <= {np.round(male_japanese_each['sod_up'],1)}")

### Female

In [ ]:
# Daily
female_japanese_day = {
    "cal_up" : 2000,
    "cal_lo" : 1950,
    "pro_up" : 50,
    "pro_lo" : 45,
    "fat_up" : 30,
    "fat_lo" : 20,
    "sod_up" : 7500,
    "sod_lo" : 7000
}
# Daily => 3 x meals
## 1 meal => Daily / 3
female_japanese_each = { k:v/3 for k, v in female_japanese_day.items() }

print("### Nutrients Conditions ###")
print(f"{np.round(female_japanese_each['cal_lo'],1)} =< calory <= {np.round(female_japanese_each['cal_up'],1)}")
print(f"{np.round(female_japanese_each['pro_lo'],1)} =< protain <= {np.round(female_japanese_each['pro_up'],1)}")
print(f"{np.round(female_japanese_each['fat_lo'],1)} =< fat <= {np.round(female_japanese_each['fat_up'],1)}")
print(f"{np.round(female_japanese_each['sod_lo'],1)} =< sodium <= {np.round(female_japanese_each['sod_up'],1)}")

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>

## Recipes for YOU

### Adult Male

In [ ]:
male_recipes = recipe_recommend(df_p2, 3, 5, male_japanese_each)
for i, j in enumerate(male_recipes):
    print(f"{i} : " + " + ".join(j))

<b>0 : Asian Barbecue Sauce  + Yam and Black Bean Burritos with Amaranth  + Mango and Red Onion Salsa </b>
<table>
    <tr>
        <td>
            <img src="https://assets.epicurious.com/photos/57c5b0b9354f42d45822eacd/16:9/w_1280,c_limit/asian-barbecue-sauce.jpg" width="300">
            <figcaption>Asian Barbecue Sauce</figcaption>
        </td>
        <td>
            <img src="https://togetherwomenrise.org/wp-content/uploads/2015/12/RECIPE-0116-Burrito.jpg" width="300">
            <figcaption>Yam and Black Bean Burritos with Amaranth</figcaption>
        </td>
        <td>
            <img src="https://cookieandkate.com/images/2015/04/fresh-mango-salsa-recipe-1.jpg" width="300">
            <figcaption>Mango and Red Onion Salsa</figcaption>
        </td>
    </tr>
</table>

### Adult Female

In [ ]:
female_recipes = recipe_recommend(df_p2, 3, 5, female_japanese_each)
for i, j in enumerate(female_recipes):
    print(f"{i} : " + " + ".join(j))

<b>0 : Yemeni Spice Rub  + Berry-Yogurt Smoothie  + Plum Freezer Jam With Cardamom and Ginger </b>
<table>
    <tr>
        <td>
            <img src="https://assets.epicurious.com/photos/5b2a903acd6f065408f725a9/4:3/w_779,h_584,c_limit/yemeni-spice-rub-recipe-062018.jpg" width="300">
            <figcaption>Yemeni Spice Rub</figcaption>
        </td>
        <td>
            <img src="https://simple-veganista.com/wp-content/uploads/2017/07/mixed-berry-yogurt-smoothie-2.jpg" width="300">
            <figcaption>Berry-Yogurt Smoothie</figcaption>
        </td>
        <td>
            <img src="https://assets.epicurious.com/photos/55f72d1ee7f5be0c7ccb6e91/5:4/w_3140,h_2512,c_limit/EP_09112015_jam_recipeimage.jpg" width="300">
            <figcaption>Plum Freezer Jam With Cardamom and Ginger</figcaption>
        </td>
    </tr>
</table>

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>

### Man in Summer

In [ ]:
male_recipes = recipe_recommend(df_p2.query("summer==1"), 3, 5, male_japanese_each)
for i, j in enumerate(male_recipes):
    print(f"{i} : " + " + ".join(j))

<b>2 : Plain Hamburgers  + Rum Punch with Passion Fruit and Lime  + Soy-Lime Dipping Sauce </b>
<table>
    <tr>
        <td>
            <img src="https://media.istockphoto.com/photos/plain-hamburger-picture-id155352091?k=20&m=155352091&s=612x612&w=0&h=DO2yEZWHr2Aik5IHgqKX7EZebgZctamoMG0Adw7LbwM=" width="300">
            <figcaption>Plain Hamburgers</figcaption>
        </td>
        <td>
            <img src="https://www.livinglocurto.com/wp-content/uploads/2019/03/Tropical-Passion-Fruit-Rum-Punch-Cocktail-Recipe.jpg" width="300">
            <figcaption>Rum Punch with Passion Fruit and Lime</figcaption>
        </td>
        <td>
            <img src="https://ucarecdn.com/096e1cd5-a853-4cf8-9169-309cf591fa1e/-/scale_crop/1280x720/center/-/quality/normal/-/format/jpeg/soy-lime-sauce.jpg" width="300">
            <figcaption>Soy-Lime Dipping Sauce</figcaption>
        </td>
    </tr>
</table>

### Man in Winter

In [ ]:
male_recipes = recipe_recommend(df_p2.query("winter==1"), 3, 5, male_japanese_each)
for i, j in enumerate(male_recipes):
    print(f"{i} : " + " + ".join(j))

<b>0 : Prunes in Wine and Armagnac  + White Bean, Pasta and Roasted Pepper Soup  + Mustard-Roasted Potatoes </b>
<table>
    <tr>
        <td>
            <img src="https://assets.marthastewart.com/styles/wmax-300/d16/prunes-mld107879/prunes-mld107879_vert.jpg?itok=1M1_dui0" width="300">
            <figcaption>Prunes in Wine and Armagnac</figcaption>
        </td>
        <td>
            <img src="https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fstatic.onecms.io%2Fwp-content%2Fuploads%2Fsites%2F44%2F2021%2F10%2F22%2Fpasta-and-white-bean-soup.jpg&q=60" width="300">
            <figcaption>White Bean, Pasta and Roasted Pepper Soup</figcaption>
        </td>
        <td>
            <img src="https://food.fnr.sndimg.com/content/dam/images/food/fullset/2012/9/19/0/BX0705H_mustard-roasted-potatoes_s4x3.jpg.rend.hgtvcom.616.462.suffix/1371611490384.jpeg" width="300">
            <figcaption>Mustard-Roasted Potatoes</figcaption>
        </td>
    </tr>
</table>

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>

### Man at Breakfast

In [ ]:
male_recipes = recipe_recommend(df_p2.query("breakfast==1"), 3, 5, male_japanese_each)
for i, j in enumerate(male_recipes):
    print(f"{i} : " + " + ".join(j))

<b>0 : Cinnamon Oat Cakes  + Strawberry Jam  + Coffee and Molasses Cured Duck Breast Bacon </b>
<table>
    <tr>
        <td>
            <img src="https://www.momendeavors.com/wp-content/uploads/2019/11/Cinnamon-Apple-Oatmeal-Cake.jpg" width="300">
            <figcaption>Cinnamon Oat Cakes</figcaption>
        </td>
        <td>
            <img src="https://thecozyapron.com/wp-content/uploads/2019/07/strawberry-jam_thecozyapron_1.jpg" width="300">
            <figcaption>Strawberry Jam</figcaption>
        </td>
        <td>
            <img src="https://i8.amplience.net/i/traeger/smoked-duck-breast-bacon_Traeger-Wood-Fired-Grills_RE_HE_M?w=1200&sm=aspect&aspect=2:1&scaleFit=poi&$poi2$" width="300">
            <figcaption>Coffee and Molasses Cured Duck Breast Bacon</figcaption>
        </td>
    </tr>
</table>

### Man with Low Calory

In [ ]:
male_recipes = recipe_recommend(df_p2[df_p2["low cal"]==1], 3, 5, male_japanese_each)
for i, j in enumerate(male_recipes):
    print(f"{i} : " + " + ".join(j))

<b>0 : Slightly Sweet Dill Refrigerator Pickles  + Homemade Chocolate Liqueur  + Shortcut Turkey Stock </b>
<table>
    <tr>
        <td>
            <img src="https://norrtable.com/wp-content/uploads/2021/10/IMG_9713_jpg-1-edited.jpg" width="300">
            <figcaption>Slightly Sweet Dill Refrigerator Pickles</figcaption>
        </td>
        <td>
            <img src="https://cookingwithmammac.com/wp-content/uploads/2019/12/Chocolate-Liqueur-Recipe-Pic.jpg" width="300">
            <figcaption>Homemade Chocolate Liqueur</figcaption>
        </td>
        <td>
            <img src="https://img.buzzfeed.com/buzzfeed-static/static/2013-09/enhanced/webdr05/26/16/enhanced-buzz-orig-14762-1380228460-30.jpg" width="300">
            <figcaption>Shortcut Turkey Stock</figcaption>
        </td>
    </tr>
</table>

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>

### Man in Busy Day

In [ ]:
male_recipes = recipe_recommend(df_p2.query("len_ingredients <= 9 and len_directions <= 3"), 3, 5, male_japanese_each)
for i, j in enumerate(male_recipes):
    print(f"{i} : " + " + ".join(j))

<b>0 : Chicken Stock  + Steamed Jasmine Rice  + Purple Potato, Sugar Snap Pea, and Mint Salad </b>
<table>
    <tr>
        <td>
            <img src="https://www.spendwithpennies.com/wp-content/uploads/2017/12/How-to-Make-Chicken-Stock-21-500x500.jpg" width="300">
            <figcaption>Chicken Stock</figcaption>
        </td>
        <td>
            <img src="https://img.taste.com.au/LR2_4Z3C/taste/2016/11/lemon-grass-jasmine-rice-5935-1.jpeg" width="300">
            <figcaption>Steamed Jasmine Rice</figcaption>
        </td>
        <td>
            <img src="https://images.ctfassets.net/uw7yiu2kuigc/7kgTMc8itJFesVCACVj47S/d57072a72e6f8d4b0490274c55e15d74/Snap-Pea-and-Potato-Salad-with-Mint-and-Honey-Lead.jpg" width="300">
            <figcaption>Purple Potato, Sugar Snap Pea, and Mint Salad</figcaption>
        </td>
    </tr>
</table>

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>

# References
<blockquote>
    <ul>
        <li><a href="https://indrag49.github.io/Numerical-Optimization/">Introduction to Mathematical Optimization</a></li>
        <li><a href="https://coin-or.github.io/pulp/">PuLP Documentation</a></li>
        <li><a href="https://www.otsuka.co.jp/cmt/nutrition/1day/">Nutrients Conditions</a></li>
    </ul>
</blockquote>

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>